In [1]:
from envs.train_env import Electric_Car
from models.baseline import BaselineModel
from models.baseline2 import BaselineModel2
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from models.tabular_qlearning import TabularQLearning
from envs.feature_engineering import DataHelper
import torch

In [2]:
def data_analysis():
    data_path_train = 'data/train.xlsx'
    df = pd.read_excel(data_path_train)

    df['Month'] = df['PRICES'].dt.month
    df['DayOfWeek'] = df['PRICES'].dt.dayofweek
    df['AvgPerDay'] = df.iloc[:, 1:25].mean(axis=1)

    monthly_avg = df.groupby('Month')['AvgPerDay'].mean()
    day_of_week_avg = df.groupby('DayOfWeek')['AvgPerDay'].mean()
    hourly_avg = df.iloc[:, 1:25].mean()

    monthly_avg.plot(kind='bar', title='Average Price per Month',edgecolor='black')
    plt.xlabel('Month')
    plt.ylabel('Average Price')
    plt.show()

    day_of_week_avg.plot(kind='bar', title='Average Price per Day of the Week',edgecolor='black')
    plt.xlabel('Day of the Week')
    plt.ylabel('Average Price')
    plt.show()

    hourly_avg.plot(kind='bar', title='Average Price per Hour',edgecolor='black')
    plt.xlabel('Hour')
    plt.ylabel('Average Price')
    plt.show()


In [3]:
# dh = DataHelper()

# def learn(env, model):
#     states, rewards, infos = [], [], []
#     truncated = False
#     terminated = False
#     while(terminated == False and truncated == False):
#         action = model.predict(env)
#         obs, reward, termination, truncation, info = env.step(action)
#         new_obs = dh.process_data(obs)

#         states.append(new_obs)
#         rewards.append(reward)
#         terminated = termination
#         truncated = truncation
#         infos.append(info)

#     return states, rewards, terminated, truncated, infos

In [ ]:
data_path_train = 'data/train.xlsx'
data_path_test = 'data/validate.xlsx'

simulations = 50
simulations_per_average = 5
learning_rate = 0.1
epsilon = 0.05
epsilon_decay = simulations
adaptive_epsilon = True
adapting_learning_rate = False
discount_rate = 0

features_Qtable = [0, 1, 2, 3, 5] #battery, price, hour, day of week, month 
bin_size = [4, 6, 7, 3, 4, 5] #bin size of features with corresponding index

model = TabularQLearning(data_path_train, discount_rate, bin_size, features_Qtable)
model.train(simulations, simulations_per_average, learning_rate, epsilon, epsilon_decay, adaptive_epsilon, adapting_learning_rate)
#model.create_Q_table()
#model.load_Qtable('model_checkpoints/Tabular1.npy')
model.visualize_rewards_and_performance()

feature 0
feature 1
feature 2
feature 3
feature 4
(3, 5, 6, 2, 3, 4)
Please wait, the algorithm is learning! The current simulation is 0
Average reward  -7267.8226518064985
Please wait, the algorithm is learning! The current simulation is 5
Average reward  -7216.760713140148
Please wait, the algorithm is learning! The current simulation is 10
Average reward  -6817.84336417359
Please wait, the algorithm is learning! The current simulation is 15
Average reward  -6169.181092806769
Please wait, the algorithm is learning! The current simulation is 20
Average reward  -5635.332941479353
Please wait, the algorithm is learning! The current simulation is 25
Average reward  -4899.024915824001


In [ ]:
#model.save_Qtable('test.npy')
actions, rewards, states, infos = model.play(data_path_test, plotting=True)

In [ ]:
print('Number of entries', model.Qtable.size)
print('Number of zeros', np.sum(model.Qtable==0))
print('Revenue', sum(rewards))

for i in range(len(features_Qtable)):
    axes = list(range(len(features_Qtable)+1))
    axes.remove(axes[i])
    sums = np.sum(model.Qtable_updates, axis=tuple(axes))
    print(sums)
    plt.bar(range(len(sums)), sums, color='green', edgecolor='black')
    plt.title('Sum of updates per cell of feature %d'%features_Qtable[i])
    plt.show()

In [ ]:
for i in range(100):
    #state : [battery_level, price, int(hour), int(day_of_week), int(day_of_year), int(month), int(year), int(self.car_is_available)]
    print('\nbattery',int(states[i][0]), '  price',int(states[i][1]), '  hour',int(states[i][2]), '  availability',int(states[i][-1]))
    print('action',infos[i])
    print('reward',rewards[i])

In [ ]:
test = [0, 49, 50]
f = np.linspace(0, 50, 5) #bin size=5, bins = 4
0-10 1
10-20 2
...
40-49.9999 4
50.1 5

q bin size, q statespace
#f[-1]+=0.1
print(f)
dig=[]
for i in test:
    dig.append(np.digitize(i, f)-1)
print(dig)